# New Section

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Importing Important modules**

In [3]:
from tensorflow.keras import layers
from tensorflow.keras import Model
import cv2

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import gc
import tensorflow as tf
%matplotlib inline


***Importing the array of cropped images as csv file***

In [4]:
df=pd.read_csv('/content/drive/MyDrive/img.csv')

Normalizing the RGB values

In [5]:
df=df/255

In [ ]:
df

storing the images as numpy array reshaping it 

In [6]:
X=np.asarray(df)
X=X.reshape(18632,100,100,3)

Importing Training Dataset

In [7]:
df_train=pd.read_csv('/content/drive/MyDrive/train.csv')

**Splitting the labels into 6 classes and encoding it**

In [8]:
df_train["labels"]=df_train["labels"].apply(lambda x:x.split(" "))

In [9]:
df_train['healthy']=0
df_train['scab']=0
df_train['complex']=0
df_train['rust']=0
df_train['frog_eye_leaf_spot']=0
df_train['powdery_mildew']=0


In [10]:
for rown in range(0,len(df_train['labels'])):
    for i in df_train['labels'][rown]:
        if i == 'healthy':
            df_train['healthy'][rown]=1
        elif i == 'scab':
            df_train['scab'][rown]=1
        elif i == 'complex':
            df_train['complex'][rown]=1
        elif i == 'rust':
            df_train['rust'][rown]=1
        elif i == 'frog_eye_leaf_spot':
            df_train['frog_eye_leaf_spot'][rown]=1
        elif i == 'powdery_mildew':
            df_train['powdery_mildew'][rown]=1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykerne

In [11]:
df_train[['healthy','scab','complex','rust','frog_eye_leaf_spot','powdery_mildew']].sum()

healthy               4624
scab                  5712
complex               2151
rust                  2077
frog_eye_leaf_spot    4352
powdery_mildew        1271
dtype: int64

Stoing the labels as 6 dimentional Tensors

In [12]:
y=np.asarray(df_train[['healthy','scab','complex','rust','frog_eye_leaf_spot','powdery_mildew']])

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y,  test_size=0.2, random_state=42)

**Importing and Using InceptionV3 pretrained model**

In [14]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

--2021-08-04 14:17:33--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.23.128, 74.125.203.128, 74.125.204.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.23.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M   184MB/s    in 0.5s    

2021-08-04 14:17:33 (184 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



In [15]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
pre_trained_model = InceptionV3(
    input_shape=(100, 100, 3), include_top=False, weights=None)
pre_trained_model.load_weights(local_weights_file)

In [16]:
for layer in pre_trained_model.layers:
  layer.trainable = False

In [17]:
last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape:', last_layer.output_shape)
last_output = last_layer.output

last layer output shape: (None, 4, 4, 768)


In [18]:
# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(128, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)
# Add a final sigmoid layer for classification
x = layers.Dense(6, activation='sigmoid')(x)



Compiling the model

In [19]:
model = Model(pre_trained_model.input, x)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                loss="binary_crossentropy",
                metrics=['accuracy'])

Training model with data

In [ ]:
model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test), batch_size=50)

Epoch 1/5
299/299 [==============================] - ETA: 0s - loss: 0.3538 - accuracy: 0.5435